In [3]:
import requests
import urllib

Onderstaande code is de basis connectie met het API, er kunnen parameters toegevoegd worden om meer gespecificeerde informatie te verkrijgen. Op deze link staan alle parameters die we toe zouden kunnen voegen:
https://developer.yummly.com/documentation

In [4]:
response = requests.get("http://api.yummly.com/v1/api/recipes?_app_id=2b082bfe&_app_key=905dfe0267500a98d67737f446479c11")

In [5]:
mediterranean_main_recipes_15min = requests.get("http://api.yummly.com/v1/api/recipes?_app_id=2b082bfe&_app_key=905dfe0267500a98d67737f446479c11&allowedCuisine[]=cuisine^cuisine-mediterranean&allowedCuisine[]=cuisine^cuisine-italian&allowedCuisine[]=cuisine^cuisine-spanish&maxTotalTimeInSeconds=600&allowedCourse[]=course^course-Main Dishes").json()
# converted to json to be sure that the right json format is being used.  
# Recipes which are either itlian, spanish or meditarean, which are a main course and take less than 15 minutes to cook. 

In [85]:
from rdflib import Graph, Namespace, URIRef, Literal, XSD, RDF

# create an RDF graph
g = Graph()



YUM = Namespace("http://yummly.com/")
FO = Namespace("http://purl.org/ontology/fo/")

g.bind('rec', YUM)
g.bind('fo', FO)

In [86]:
print mediterranean_main_recipes_15min['matches'][0].keys()

[u'flavors', u'rating', u'totalTimeInSeconds', u'ingredients', u'smallImageUrls', u'sourceDisplayName', u'recipeName', u'attributes', u'id', u'imageUrlsBySize']


In [87]:
ingredient_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    for ingredient in recipe['ingredients']:
        ingredient_list.append(ingredient)
        
recipe_name_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    recipe_name_list.append(recipe['recipeName'])

cooking_time_list = []
for recipe in mediterranean_main_recipes_15min['matches']:
    cooking_time_list.append(recipe['totalTimeInSeconds'])

In [88]:
for ingredient in ingredient_list:
    ingredientNoSpace = ingredient.replace(" ", "_")
    g.add((YUM[ingredientNoSpace], RDF.type, FO["Ingredient"]))

In [89]:
print g.serialize(format='turtle')

@prefix fo: <http://purl.org/ontology/fo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rec: <http://yummly.com/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

rec:Italian_bread a fo:Ingredient .

rec:angel_hair a fo:Ingredient .

rec:baby_spinach a fo:Ingredient .

rec:basil_leaves a fo:Ingredient .

rec:black_pepper a fo:Ingredient .

rec:butter a fo:Ingredient .

rec:cherry_tomatoes a fo:Ingredient .

rec:chicken_breasts a fo:Ingredient .

<http://yummly.com/clove_garlic,_fine_chop> a fo:Ingredient .

rec:cooking_spray a fo:Ingredient .

rec:crushed_red_pepper_flakes a fo:Ingredient .

rec:cumin a fo:Ingredient .

rec:diced_tomatoes a fo:Ingredient .

rec:eggplant a fo:Ingredient .

rec:extra-virgin_olive_oil a fo:Ingredient .

rec:feta_cheese a fo:Ingredient .

rec:finely_chopped_fresh_parsley a fo:Ingredient .

rec:fresh_basil a fo:Ingredi

In [90]:
for recipe in recipe_name_list:
    recipeNoSpace = recipe.replace(" ", "_")
    cleanRecipe = recipeNoSpace.replace("{","(")
    cleanRecipe = cleanRecipe.replace("}",")")
    g.add((YUM[cleanRecipe], RDF.type, FO["Recipe"]))

In [91]:
print g.serialize(format='turtle')

@prefix fo: <http://purl.org/ontology/fo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rec: <http://yummly.com/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://yummly.com/(5-ingredient)_Italian_Caprese_Pasta> a fo:Recipe .

rec:Baba_Ghanoush_Topped_with_Lamb_and_Yogurt a fo:Recipe .

rec:Braised_Italian_Pot_Roast a fo:Recipe .

rec:Easy_Garlic_and_Lemon_Shrimp a fo:Recipe .

rec:Easy_Peasy_Skillet_Lasagna a fo:Recipe .

rec:Italian_Chicken_Panini a fo:Recipe .

rec:Italian_bread a fo:Ingredient .

<http://yummly.com/Macaroni_with_Greens,_Lemon,_and_Parmesan> a fo:Recipe .

rec:Mediterranean_Pork_Chops a fo:Recipe .

rec:Spaghetti_Aglio_e_Olio a fo:Recipe .

rec:Spinach_and_Ricotta_with_Shells a fo:Recipe .

rec:angel_hair a fo:Ingredient .

rec:baby_spinach a fo:Ingredient .

rec:basil_leaves a fo:Ingredient .

rec:black_pepper a fo:Ing